In [1]:
import xarray
import numpy
import os
os.environ["ECCODES_DIR"] = "/home/nen/micromamba/envs/eccodes_env"

from meteodatalab import data_source, grib_decoder
import eccodes
print("ecCodes version:", eccodes.codes_get_api_version())


ecCodes version: 2.35.1


In [2]:
source = data_source.FileDataSource(datafiles=["precip_20250122_06.grib"])
source

FileDataSource(request_template={}, datafiles=['precip_20250122_06.grib'])

In [3]:
request = {"param":["TOT_PREC"]}
ds_in = grib_decoder.load(source,request)
ds_in

{'TOT_PREC': <xarray.DataArray (eps: 10, ref_time: 1, lead_time: 4, y: 786, x: 1170)> Size: 147MB
 array([[[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           ...,
           [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
           [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],
 
          [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
 